# 预训练BERT

In [ ]:
import os
import random
import time
import torch
from torch import nn
import collections
import math
import matplotlib.pyplot as plt
from IPython import display

# ===============================
# 纯原生实现：替代d2l库的所有功能
# ===============================

def tokenize(lines, token='word'):
    """
    将文本行分词为单词或字符
    
    参数：
        lines: 字符串列表，每个字符串是一行文本
        token: 'word' 或 'char'，分词粒度
    
    返回：
        tokens: 二维列表，每个元素是一行的词元列表
    """
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        raise ValueError(f"未知的token类型: {token}")


class Vocab:
    """
    词表类：实现词元到索引的双向映射
    
    特殊词元：
        <unk>: 未知词元，索引为0
        其他reserved_tokens按顺序添加
    """
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """
        参数：
            tokens: 二维列表，所有句子的词元
            min_freq: 最小词频，低于此频率的词元将被忽略
            reserved_tokens: 预留的特殊词元列表
        """
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # 统计词频
        counter = collections.Counter()
        for line in tokens:
            for token in line:
                counter[token] += 1
        
        # 按词频排序
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # 构建词表
        # <unk>的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        
        # 添加满足最小词频的词元
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        """将词元转换为索引"""
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.token_to_idx['<unk>'])
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        """将索引转换为词元"""
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[idx] for idx in indices]


def get_tokens_and_segments(tokens_a, tokens_b=None):
    """
    构造BERT的输入格式
    
    格式：<cls> 句子A <sep> [句子B <sep>]
    
    参数：
        tokens_a: 句子A的词元列表
        tokens_b: 句子B的词元列表（可选）
    
    返回：
        tokens: 完整的词元列表
        segments: 段落标识列表（0表示句子A，1表示句子B）
    """
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 句子A部分的segment为0
    segments = [0] * (len(tokens_a) + 2)
    
    if tokens_b is not None:
        tokens = tokens + tokens_b + ['<sep>']
        # 句子B部分的segment为1
        segments = segments + [1] * (len(tokens_b) + 1)
    
    return tokens, segments


def masked_softmax(X, valid_lens):
    """
    带掩码的softmax操作
    
    参数：
        X: 形状为(batch_size, num_queries, num_keys)的张量
        valid_lens: 有效长度，形状为(batch_size,)或(batch_size, num_queries)
    
    返回：
        softmax后的注意力权重，无效位置的权重为0
    """
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            # valid_lens形状为(batch_size,)，扩展到每个query
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            # valid_lens形状为(batch_size, num_queries)
            valid_lens = valid_lens.reshape(-1)
        
        # 将X展平为(batch_size * num_queries, num_keys)
        X = X.reshape(-1, shape[-1])
        
        # 创建掩码：超过有效长度的位置设为很大的负数
        max_len = X.shape[1]
        mask = torch.arange(max_len, dtype=torch.float32, device=X.device)[None, :] < valid_lens[:, None]
        X[~mask] = -1e6
        
        return nn.functional.softmax(X.reshape(shape), dim=-1)


class DotProductAttention(nn.Module):
    """
    缩放点积注意力
    
    公式：Attention(Q, K, V) = softmax(QK^T / sqrt(d_k)) V
    """
    def __init__(self, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, queries, keys, values, valid_lens=None):
        """
        参数：
            queries: (batch_size, num_queries, d)
            keys: (batch_size, num_keys, d)
            values: (batch_size, num_keys, d_v)
            valid_lens: (batch_size,) 或 (batch_size, num_queries)
        
        返回：
            output: (batch_size, num_queries, d_v)
        """
        d = queries.shape[-1]
        # 计算注意力分数: (batch_size, num_queries, num_keys)
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        # 应用掩码softmax
        self.attention_weights = masked_softmax(scores, valid_lens)
        # 计算输出
        return torch.bmm(self.dropout(self.attention_weights), values)


def try_all_gpus():
    """获取所有可用的GPU，如果没有GPU则返回CPU"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


class Timer:
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start_time = None
    
    def start(self):
        """启动计时器"""
        self.start_time = time.time()
    
    def stop(self):
        """停止计时器并记录时间"""
        self.times.append(time.time() - self.start_time)
        return self.times[-1]
    
    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)
    
    def sum(self):
        """返回总时间"""
        return sum(self.times)
    
    def cumsum(self):
        """返回累计时间"""
        return list(itertools.accumulate(self.times))


class Animator:
    """在动画中绘制数据（用于训练可视化）"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        if legend is None:
            legend = []
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes]
        self.config_axes = lambda: self.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
    
    def set_axes(self, ax, xlabel, ylabel, xlim, ylim, xscale, yscale, legend):
        """设置matplotlib的轴"""
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_xscale(xscale)
        ax.set_yscale(yscale)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        if legend:
            ax.legend(legend)
        ax.grid()
    
    def add(self, x, y):
        """向图表中添加数据点"""
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)


class Accumulator:
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]
    
    def reset(self):
        self.data = [0.0] * len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
def _read_wiki(data_dir):
    """
    读取WikiText-2数据集并预处理成段落格式
    
    WikiText-2数据集结构：
    - 每一行是一个段落
    - 段落内的句子用 " . " 分隔
    
    返回值结构：
    paragraphs = [
        ["sentence1", "sentence2", ...],  # 段落1
        ["sentence1", "sentence2", ...],  # 段落2
        ...
    ]
    """
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    
    paragraphs = []
    for line in lines:
        # 按 " . " 分割成句子列表
        sentences = line.strip().lower().split(' . ')
        # 只保留包含至少2个句子的段落（NSP任务需要句子对）
        if len(sentences) >= 2:
            paragraphs.append(sentences)
    
    # 打乱段落顺序，增加数据随机性
    random.shuffle(paragraphs)
    return paragraphs

## 第一步：读取和预处理数据

WikiText-2是一个从Wikipedia提取的语言建模数据集，包含约200万个词元。

In [ ]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    """
    为NSP任务生成句子对
    
    参数：
        sentence: 当前句子（句子A）
        next_sentence: 原本的下一个句子
        paragraphs: 所有段落，用于采样负样本
    
    返回：
        sentence: 句子A
        next_sentence: 句子B（可能是真实下一句或随机句子）
        is_next: 布尔值，True表示是真实的下一句
    
    策略：50%概率返回真实下一句，50%概率返回随机句子
    """
    if random.random() < 0.5:
        # 正样本：使用真实的下一个句子
        is_next = True
    else:
        # 负样本：从随机段落中随机选一个句子
        # random.choice(paragraphs) -> 随机选一个段落
        # random.choice(...) -> 从该段落中随机选一个句子
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

## 第二步：生成下一句预测(NSP)任务数据

NSP任务的目标是判断两个句子是否相邻。训练数据构造方式：
- **正样本(is_next=True)**：取相邻的两个句子
- **负样本(is_next=False)**：取一个句子 + 随机句子

BERT输入格式：`<cls> 句子A <sep> 句子B <sep>`

In [ ]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    """
    从一个段落生成所有NSP训练样本
    
    参数：
        paragraph: 当前段落（句子列表，每个句子是词元列表）
        paragraphs: 所有段落（用于采样负样本）
        vocab: 词表（此函数中未使用，但保持接口一致）
        max_len: 最大序列长度
    
    返回：
        nsp_data_from_paragraph: [(tokens, segments, is_next), ...]
        - tokens: ['<cls>', 词1, 词2, ..., '<sep>', 词1, 词2, ..., '<sep>']
        - segments: [0, 0, ..., 0, 1, 1, ..., 1] 表示属于句子A还是句子B
        - is_next: True/False
    """
    nsp_data_from_paragraph = []
    
    # 遍历段落中的相邻句子对
    for i in range(len(paragraph) - 1):
        # 生成句子对（可能是正样本或负样本）
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        
        # 检查长度：tokens_a + tokens_b + 3个特殊词元(<cls>, <sep>, <sep>)
        # 如果超过max_len则跳过这个样本
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        
        # 使用自定义函数构造BERT输入格式（替代d2l.get_tokens_and_segments）
        # tokens: ['<cls>'] + tokens_a + ['<sep>'] + tokens_b + ['<sep>']
        # segments: [0]*len(句子A部分) + [1]*len(句子B部分)
        tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    
    return nsp_data_from_paragraph

In [ ]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds, vocab):
    """
    对tokens进行遮蔽处理，生成MLM任务的输入和标签
    
    参数：
        tokens: 原始词元列表 ['<cls>', 'hello', 'world', '<sep>', ...]
        candidate_pred_positions: 可以被遮蔽的位置索引列表（不包括特殊词元）
        num_mlm_preds: 需要遮蔽的词元数量（约15%）
        vocab: 词表（用于随机替换时采样）
    
    返回：
        mlm_input_tokens: 遮蔽后的词元列表（用于模型输入）
        pred_positions_and_labels: [(位置, 原始词元), ...] 用于计算损失
    """
    # 复制一份tokens，避免修改原始数据
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    
    # 随机打乱候选位置，然后选取前num_mlm_preds个
    random.shuffle(candidate_pred_positions)
    
    for mlm_pred_position in candidate_pred_positions:
        # 已选够需要预测的数量，退出循环
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        
        masked_token = None
        rand = random.random()
        
        if rand < 0.8:
            # 80%概率：替换为<mask>词元
            masked_token = '<mask>'
        elif rand < 0.9:
            # 10%概率：保持原词不变（但仍然要预测）
            masked_token = tokens[mlm_pred_position]
        else:
            # 10%概率：替换为词表中的随机词
            masked_token = random.choice(vocab.idx_to_token)
        
        # 在输入中进行替换
        mlm_input_tokens[mlm_pred_position] = masked_token
        # 记录位置和原始词元（作为预测标签）
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    
    return mlm_input_tokens, pred_positions_and_labels

## 第三步：生成遮蔽语言模型(MLM)任务数据

MLM任务的核心思想是"完形填空"：随机遮蔽一些词，让模型根据上下文预测被遮蔽的词。

### 遮蔽策略（针对被选中的15%词元）：
| 概率 | 处理方式 | 原因 |
|------|----------|------|
| 80% | 替换为`<mask>` | 主要的遮蔽方式 |
| 10% | 保持不变 | 避免模型认为`<mask>`一定要预测 |
| 10% | 替换为随机词 | 增加鲁棒性 |

这种策略可以避免预训练和微调之间的不匹配问题。

In [ ]:
def _get_mlm_data_from_tokens(tokens, vocab):
    """
    从词元序列生成MLM任务的完整数据
    
    参数：
        tokens: 词元列表 ['<cls>', 'the', 'cat', '<sep>', 'sat', '<sep>']
        vocab: 词表对象
    
    返回：
        token_ids: 遮蔽后的词元ID列表（模型输入）
        pred_positions: 被遮蔽的位置列表
        mlm_pred_label_ids: 被遮蔽位置的原始词元ID（预测标签）
    
    示例：
        输入tokens: ['<cls>', 'the', 'cat', 'sat', '<sep>']
        假设遮蔽位置2(cat)和3(sat)
        输出:
            token_ids: [cls_id, the_id, mask_id, random_id, sep_id]
            pred_positions: [2, 3]
            mlm_pred_label_ids: [cat_id, sat_id]
    """
    # 找出所有可以被遮蔽的位置（排除特殊词元）
    candidate_pred_positions = []
    for i, token in enumerate(tokens):
        # <cls>和<sep>是特殊词元，不参与MLM预测
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    
    # 计算需要遮蔽的词元数量：约15%，但至少1个
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    
    # 执行遮蔽操作
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    
    # 按位置排序，保证顺序一致性
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    
    # 分离位置和标签
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    
    # 将词元转换为ID（vocab[tokens]会调用vocab的__getitem__方法）
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

In [ ]:
def _pad_bert_inputs(examples, max_len, vocab):
    """
    将所有样本填充到相同长度，转换为PyTorch张量
    
    参数：
        examples: [(token_ids, pred_positions, mlm_labels, segments, is_next), ...]
        max_len: 最大序列长度
        vocab: 词表对象
    
    返回（7个列表，每个列表包含所有样本的对应数据）：
        all_token_ids: 填充后的输入序列 [batch, max_len]
        all_segments: 段落标识 [batch, max_len]
        valid_lens: 有效长度（不含<pad>）[batch]
        all_pred_positions: MLM预测位置 [batch, max_num_mlm_preds]
        all_mlm_weights: MLM损失权重 [batch, max_num_mlm_preds]
        all_mlm_labels: MLM预测标签 [batch, max_num_mlm_preds]
        nsp_labels: NSP标签 [batch]
    """
    # MLM预测的最大数量 = 序列长度 * 15%
    max_num_mlm_preds = round(max_len * 0.15)
    
    # 初始化存储列表
    all_token_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    
    for (token_ids, pred_positions, mlm_pred_label_ids, segments, is_next) in examples:
        # ===== 1. 填充token_ids到max_len =====
        # 在序列末尾添加<pad>词元
        padding_len = max_len - len(token_ids)
        all_token_ids.append(torch.tensor(
            token_ids + [vocab['<pad>']] * padding_len, 
            dtype=torch.long))
        
        # ===== 2. 填充segments到max_len =====
        # 填充部分的segment标识为0
        all_segments.append(torch.tensor(
            segments + [0] * padding_len, 
            dtype=torch.long))
        
        # ===== 3. 记录有效长度（用于attention mask）=====
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        
        # ===== 4. 填充pred_positions到max_num_mlm_preds =====
        mlm_padding_len = max_num_mlm_preds - len(pred_positions)
        all_pred_positions.append(torch.tensor(
            pred_positions + [0] * mlm_padding_len, 
            dtype=torch.long))
        
        # ===== 5. 设置mlm_weights（真实预测位置权重为1，填充位置权重为0）=====
        # 这样在计算损失时，填充位置的损失会被过滤掉
        all_mlm_weights.append(torch.tensor(
            [1.0] * len(mlm_pred_label_ids) + [0.0] * mlm_padding_len,
            dtype=torch.float32))
        
        # ===== 6. 填充mlm_labels =====
        all_mlm_labels.append(torch.tensor(
            mlm_pred_label_ids + [0] * mlm_padding_len, 
            dtype=torch.long))
        
        # ===== 7. NSP标签（True->1, False->0）=====
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)

## 第四步：填充数据到固定长度

由于不同样本的序列长度不同，需要将它们填充到相同长度才能批量处理。

### 需要填充的内容：
| 数据 | 填充值 | 说明 |
|------|--------|------|
| token_ids | `<pad>` | 输入序列填充 |
| segments | 0 | 段落标识填充 |
| pred_positions | 0 | MLM预测位置填充 |
| mlm_weights | 0.0 | 填充位置的损失权重为0 |
| mlm_labels | 0 | MLM标签填充 |

In [ ]:
class _WikiTextDataset(torch.utils.data.Dataset):
    """
    WikiText-2 BERT预训练数据集
    
    数据处理流程：
    1. 分词：将句子字符串转换为词元列表
    2. 构建词表：基于所有句子构建词表
    3. 生成NSP数据：为每个段落生成句子对
    4. 生成MLM数据：对每个句子对进行遮蔽处理
    5. 填充对齐：将所有样本填充到相同长度
    """
    
    def __init__(self, paragraphs, max_len):
        """
        参数：
            paragraphs: 段落列表，每个段落是句子字符串列表
                       [["sentence1", "sentence2"], ["sentence3", "sentence4"], ...]
            max_len: 最大序列长度
        """
        # ===== 步骤1: 分词 =====
        # 将每个句子从字符串转换为词元列表
        # 输入: [["hello world", "foo bar"], ...]
        # 输出: [[["hello", "world"], ["foo", "bar"]], ...]
        # 使用自定义tokenize函数（替代d2l.tokenize）
        paragraphs = [tokenize(paragraph, token='word') 
                      for paragraph in paragraphs]
        
        # ===== 步骤2: 构建词表 =====
        # 将所有句子展平成一个列表用于构建词表
        sentences = [sentence for paragraph in paragraphs 
                     for sentence in paragraph]
        # min_freq=5: 词频少于5的词会被替换为<unk>
        # reserved_tokens: 预留的特殊词元
        # 使用自定义Vocab类（替代d2l.Vocab）
        self.vocab = Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>',   # 填充词元
            '<mask>',  # MLM遮蔽词元
            '<cls>',   # 句子开头词元
            '<sep>'    # 句子分隔词元
        ])
        
        # ===== 步骤3: 生成NSP数据 =====
        # 为每个段落生成所有可能的句子对
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        
        # ===== 步骤4: 生成MLM数据 =====
        # 对每个句子对进行遮蔽处理
        # 输入: (tokens, segments, is_next)
        # 输出: (token_ids, pred_positions, mlm_labels, segments, is_next)
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                     + (segments, is_next))
                    for tokens, segments, is_next in examples]
        
        # ===== 步骤5: 填充到固定长度 =====
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        """返回第idx个样本的所有数据"""
        return (self.all_token_ids[idx],      # 输入token IDs
                self.all_segments[idx],        # 段落标识(0或1)
                self.valid_lens[idx],          # 有效长度
                self.all_pred_positions[idx],  # MLM预测位置
                self.all_mlm_weights[idx],     # MLM损失权重
                self.all_mlm_labels[idx],      # MLM预测标签
                self.nsp_labels[idx])          # NSP标签(0或1)

    def __len__(self):
        """返回数据集大小"""
        return len(self.all_token_ids)

## 第五步：创建PyTorch Dataset

将所有数据处理步骤整合到一个Dataset类中，便于与DataLoader配合使用。

In [ ]:
def load_data_wiki(batch_size, max_len):
    """
    加载WikiText-2数据集用于BERT预训练
    
    参数：
        batch_size: 批量大小
        max_len: 最大序列长度（包含特殊词元）
    
    返回：
        train_iter: 训练数据迭代器
        vocab: 词表对象
    """
    # 使用本地已解压的数据目录
    data_dir = '../data/wikitext-2'
    
    # 读取并预处理数据
    paragraphs = _read_wiki(data_dir)
    
    # 创建数据集
    train_set = _WikiTextDataset(paragraphs, max_len)
    
    # 创建数据加载器
    # 注意：在Jupyter notebook中需要设置num_workers=0
    # 因为notebook中定义的类无法被pickle序列化用于多进程
    train_iter = torch.utils.data.DataLoader(
        train_set, 
        batch_size=batch_size,
        shuffle=True,
        num_workers=0  # notebook环境下必须为0
    )
    
    return train_iter, train_set.vocab

## 第六步：数据加载器

将数据集包装成DataLoader，支持批量加载和多进程预取。

In [ ]:
# 设置超参数
batch_size = 512  # 批量大小
max_len = 64      # 最大序列长度

# 加载数据
train_iter, vocab = load_data_wiki(batch_size, max_len)

# 查看第一个batch的形状
for (tokens_X, segments_X, valid_lens_x, pred_positions_X, 
     mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    
    print(f"tokens_X:        {tokens_X.shape}")        # [512, 64] - 输入序列
    print(f"segments_X:      {segments_X.shape}")      # [512, 64] - 段落标识
    print(f"valid_lens_x:    {valid_lens_x.shape}")    # [512] - 有效长度
    print(f"pred_positions_X:{pred_positions_X.shape}")# [512, 10] - MLM位置 (64*0.15≈10)
    print(f"mlm_weights_X:   {mlm_weights_X.shape}")   # [512, 10] - MLM权重
    print(f"mlm_Y:           {mlm_Y.shape}")           # [512, 10] - MLM标签
    print(f"nsp_y:           {nsp_y.shape}")           # [512] - NSP标签
    break

## 测试数据加载

验证数据加载器是否正常工作，并查看各张量的形状。

### 输出形状说明：
| 张量 | 形状 | 说明 |
|------|------|------|
| tokens_X | [batch, max_len] | 输入token IDs |
| segments_X | [batch, max_len] | 段落标识 |
| valid_lens_x | [batch] | 每个样本的有效长度 |
| pred_positions_X | [batch, max_mlm_preds] | MLM预测位置 |
| mlm_weights_X | [batch, max_mlm_preds] | MLM损失权重 |
| mlm_Y | [batch, max_mlm_preds] | MLM预测标签 |
| nsp_y | [batch] | NSP标签 |

In [ ]:
# 查看词表大小
print(f"词表大小: {len(vocab)}")

In [ ]:
# 该cell中的导入已经在第一个cell中完成，此处留空
# 所有d2l依赖已替换为纯原生实现

In [ ]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

In [ ]:
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者"键－值"对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者"键－值"对的个数，num_heads， num_hiddens/num_heads)
    X= X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    # 输出X的形状:(batch_size，num_heads，查询或者"键－值"对的个数, num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)
    # 最终输出的形状:(batch_size*num_heads,查询或者"键－值"对的个数, num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)


class MultiHeaderAttention(nn.Module):
    """
    多头注意力机制
    
    使用自定义的DotProductAttention（替代d2l.DotProductAttention）
    """
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False) -> None:
        super().__init__()
        self.num_heads = num_heads
        # 使用自定义的DotProductAttention
        self.attention = DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    
    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者"键－值"对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者"键－值"对的个数，num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        
        if valid_lens is not None:
            # 将 valid_lens 重复 num_heads 次，因为每个注意力头都需要独立的 valid_lens
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0
            )
        
        output = self.attention(queries, keys, values, valid_lens)
        
        output_cat = transpose_output(output, num_heads=self.num_heads)
        return self.W_o(output_cat)


class AddNorm(nn.Module):
    """残差连接 + 层归一化"""
    def __init__(self, normalized_shape, dropout) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)


class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs) -> None:
        super().__init__()
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
    
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))


class EncoderBlock(nn.Module):
    """Transformer编码器块"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        # 使用自定义的MultiHeaderAttention
        self.attention = MultiHeaderAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            bias=use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [ ]:
# get_tokens_and_segments函数已经在第一个cell中定义
# 此处留空，避免重复定义

In [ ]:
class BertEncoder(nn.Module):
    """
    BertEncoder
    segment表示"句子片段类型embedding"。
    在BERT中，输入通常是两段文本拼接，例如句子A和句子B。
    segment用于区分不同的句子（例如A为0，B为1），以便模型能够知道某个token属于哪一部分。

    输入是2，代表segment可以取两种类型（0或1）：0表示第一个句子片段，1表示第二个句子片段。
    如果只输入单句任务，全部segment为0；如果是句子对任务，根据分割点设置为0和1。
    """
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768) -> None:
        super().__init__()
        self.token_embeding = nn.Embedding(vocab_size, num_hiddens)
        # segment_embeding输入2，代表两种类型（句子1和句子2：0或1）
        self.segment_embeding = nn.Embedding(2, num_hiddens)
        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", EncoderBlock(key_size, query_size, value_size, num_hiddens, norm_shape,
                                                      ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 位置编码，shape=[1, max_len, num_hiddens]，可学习
        self.pos_embeding = nn.Parameter(torch.randn(1, max_len, num_hiddens))
    
    def forward(self, token, segment, valid_lens):
        """
        token: 词索引序列，[batch, seq_len]
        segment: 句子片段类型，[batch, seq_len]，值为0或1
        valid_lens: 有效长度
        """
        # token embedding + segment embedding
        X = self.token_embeding(token) + self.segment_embeding(segment)
        # 加上可学习的位置编码
        X = X + self.pos_embeding.data[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

In [ ]:
class MaskLM(nn.Module):
    def __init__(self, vocab_size, num_hiddens, num_inputs=768) -> None:
        """
        Masked Language Model（MLM）模块。

        参数说明：
        vocab_size: 词表大小，输出类别数（即预测每个位置对应的词汇表token）。
        num_hiddens: 隐藏层的维度。
        num_inputs: 输入特征的维度，通常等于BERT编码器输出的隐藏单元数，默认768。
        """
        super().__init__()
        # 构造一个MLP（多层感知机），输入num_inputs维，经过隐藏层后输出vocab_size维的预测。
        self.mlp = nn.Sequential(
            nn.Linear(num_inputs, num_hiddens), # 线性变换到隐藏层
            nn.ReLU(),                          # 激活函数
            nn.LayerNorm(num_hiddens),          # 层归一化
            nn.Linear(num_hiddens, vocab_size)  # 映射到vocab_size，为softmax前的logits
        )
    
    def forward(self, X, pred_positions):
        """
        前向传播。

        参数：
        X: 经过BERT encoder后的表示，形状为 [batch_size, seq_len, hidden_dim]，
           表示每个token的上下文表示。
        pred_positions: 要预测的masked位置索引，形状为 [batch_size, num_pred_positions]，
                        每行是一个样本要预测的位置列表。

        返回：
        mlm_Y_hat: 每个被mask位置的预测结果，
                   形状为 [batch_size, num_pred_positions, vocab_size]。
        """
        # 1. 得到每个样本需要预测的token数量
        num_pred_positions = pred_positions.shape[1]
        # 2. 将pred_positions展平为一维，便于统一索引
        pred_positions_flat = pred_positions.reshape(-1)  # 长度为batch_size * num_pred_positions

        batch_size = X.shape[0]
        # 3. 构造一个batch索引。例如batch_size=2, num_pred_positions=3时，得到[0,0,0,1,1,1]
        batch_idx = torch.arange(0, batch_size).repeat_interleave(num_pred_positions)
        # 这样(X[batch_idx, pred_positions_flat])就取出所有需要mask的token的表示

        # 4. 按指定位置收集得到被mask位置的上下文表示，形状为 [batch_size * num_pred_positions, hidden_dim]
        masked_X = X[batch_idx, pred_positions_flat]
        # 5. 恢复成 [batch_size, num_pred_positions, hidden_dim] 的形式
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1))

        # 6. 通过MLP变换，每个位置最终输出vocab_size维，对应softmax前的logits
        mlm_Y_hat = self.mlp(masked_X)

        # 7. 输出，形状为 [batch_size, num_pred_positions, vocab_size]
        return mlm_Y_hat

In [ ]:
class NextSentencePred(nn.Module):
    def __init__(self, num_inputs) -> None:
        super().__init__()
        self.output = nn.Linear(num_inputs, 2)
    
    def forward(self, X):
        return self.output(X)

In [ ]:
#@save
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,
                 nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BertEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None,
                pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :]))
        return encoded_X, mlm_Y_hat, nsp_Y_hat

In [ ]:
# 创建BERT模型
# max_len设置为64（与数据集的max_len一致）
net = BERTModel(len(vocab), num_hiddens=128, norm_shape=[128],
                ffn_num_input=128, ffn_num_hiddens=256, num_heads=2,
                num_layers=2, dropout=0.2, key_size=128, query_size=128,
                value_size=128, hid_in_features=128, mlm_in_features=128,
                nsp_in_features=128, max_len=64)

# 使用自定义的try_all_gpus函数（替代d2l.try_all_gpus）
devices = try_all_gpus()
loss = nn.CrossEntropyLoss()

In [ ]:
#@save
def _get_batch_loss_bert(net, loss, vocab_size, tokens_X,
                         segments_X, valid_lens_x,
                         pred_positions_X, mlm_weights_X,
                         mlm_Y, nsp_y):
    # 前向传播
    _, mlm_Y_hat, nsp_Y_hat = net(tokens_X, segments_X,
                                  valid_lens_x.reshape(-1),
                                  pred_positions_X)
    # 计算遮蔽语言模型损失
    mlm_l = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y.reshape(-1)) *\
    mlm_weights_X.reshape(-1, 1)
    mlm_l = mlm_l.sum() / (mlm_weights_X.sum() + 1e-8)
    # 计算下一句子预测任务的损失
    nsp_l = loss(nsp_Y_hat, nsp_y)
    l = mlm_l + nsp_l
    return mlm_l, nsp_l, l

In [ ]:
def train_bert(train_iter, net, loss, vocab_size, devices, num_steps):
    """
    训练BERT模型
    
    使用自定义的Timer、Animator、Accumulator（替代d2l版本）
    """
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    trainer = torch.optim.Adam(net.parameters(), lr=0.01)
    step = 0
    timer = Timer()  # 使用自定义Timer
    animator = Animator(xlabel='step', ylabel='loss',
                        xlim=[1, num_steps], legend=['mlm', 'nsp'])  # 使用自定义Animator
    # 遮蔽语言模型损失的和，下一句预测任务损失的和，句子对的数量，计数
    metric = Accumulator(4)  # 使用自定义Accumulator
    num_steps_reached = False
    while step < num_steps and not num_steps_reached:
        for tokens_X, segments_X, valid_lens_x, pred_positions_X,\
            mlm_weights_X, mlm_Y, nsp_y in train_iter:
            tokens_X = tokens_X.to(devices[0])
            segments_X = segments_X.to(devices[0])
            valid_lens_x = valid_lens_x.to(devices[0])
            pred_positions_X = pred_positions_X.to(devices[0])
            mlm_weights_X = mlm_weights_X.to(devices[0])
            mlm_Y, nsp_y = mlm_Y.to(devices[0]), nsp_y.to(devices[0])
            trainer.zero_grad()
            timer.start()
            mlm_l, nsp_l, l = _get_batch_loss_bert(
                net, loss, vocab_size, tokens_X, segments_X, valid_lens_x,
                pred_positions_X, mlm_weights_X, mlm_Y, nsp_y)
            l.backward()
            trainer.step()
            metric.add(mlm_l, nsp_l, tokens_X.shape[0], 1)
            timer.stop()
            animator.add(step + 1,
                         (metric[0] / metric[3], metric[1] / metric[3]))
            step += 1
            if step == num_steps:
                num_steps_reached = True
                break

    print(f'MLM loss {metric[0] / metric[3]:.3f}, '
          f'NSP loss {metric[1] / metric[3]:.3f}')
    print(f'{metric[2] / timer.sum():.1f} sentence pairs/sec on '
          f'{str(devices)}')

In [ ]:
train_bert(train_iter, net, loss, len(vocab), devices, 50)

In [ ]:
def get_bert_encoding(net, tokens_a, tokens_b=None):
    """
    获取BERT编码
    
    使用自定义的get_tokens_and_segments函数（替代d2l版本）
    """
    tokens, segments = get_tokens_and_segments(tokens_a, tokens_b)
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments = torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoded_X

In [ ]:
# 下面，我们以"a crane is flying"这个句子作为输入，获取其BERT编码
tokens_a = ['a', 'crane', 'is', 'flying']
encoded_text = get_bert_encoding(net, tokens_a)

# encoded_text的形状为(1, 7, 隐藏层维数)，7表示句子的tokens数量（包括特殊token，如<cls>、<sep>等）
print("encoded_text.shape:", encoded_text.shape)

# [CLS]位置的输出表示整个句子的语义，用于句子级任务
encoded_text_cls = encoded_text[:, 0, :]
print("encoded_text_cls.shape:", encoded_text_cls.shape)

# "crane"单词在tokens中的位置是2，获取其BERT编码
encoded_text_crane = encoded_text[:, 2, :]
print("encoded_text_crane向量的前3维:", encoded_text_crane[0][:3])

In [ ]:
tokens_a, tokens_b = ['a', 'crane', 'driver', 'came'], ['he', 'just', 'left']
encoded_pair = get_bert_encoding(net, tokens_a, tokens_b)
# 词元：'<cls>','a','crane','driver','came','<sep>','he','just',
# 'left','<sep>'
encoded_pair_cls = encoded_pair[:, 0, :]
encoded_pair_crane = encoded_pair[:, 2, :]
encoded_pair.shape, encoded_pair_cls.shape, encoded_pair_crane[0][:3]